# 3: Assessing model performance and applicability

Author: Daniel Lusk

## Imports

In [ ]:
import pandas as pd
from utils.models import TrainedSet

%load_ext autoreload
%autoreload 2

## Area of Applicability (AOA)

## Model performance

Get the training run data from the results dataframe

In [ ]:
# Load a model
results_df = pd.read_csv("./results/training_results.csv").copy()
training_id = "2023-07-21_17-01-49"
results = results_df.loc[(results_df["Run ID"] == training_id)].sort_values(by="Test r-squared", ascending=False)

Load the best performing model

In [ ]:
row = results.iloc[0]
trained = TrainedSet.from_results_row(row)

Load unseen predictor data

In [ ]:
import os
import geopandas as gpd
from utils.models import Prediction

new_df_fn = os.path.join("data/collections", row["collection"])
new_df = gpd.read_feather(new_df_fn)

new_df_imputed_fn = "data/collections/MOD09GA.061_ISRIC_soil_WC_BIO_VODCA_0.5_deg_nan-strat=all_imputed.feather"
new_df_imputed = gpd.read_feather(new_df_imputed_fn)

pred = Prediction(trained, new_df, new_df_imputed)

Create new dataframe with predictions, masked according to the Area of Applicability (may take a while for AoA calculation)

In [ ]:
d = {"trait": pred.predictions}
predictions_df = gpd.GeoDataFrame(d, geometry=new_df.geometry, crs=new_df.crs, index=new_df.index)
predictions_df["masked_aoa"] = predictions_df["trait"].where(pred.aoa.df["AOA"] == 1)

# CV Coefficient of Variation